In [35]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, BatchNormalization, Dropout, Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras import regularizers
from keras.src.initializers import HeNormal

In [36]:
# Hyperparameters 
num_epochs = 100
batch_size = 64

In [37]:
def load_data(n, data_path=None, label_path=None):
    if data_path and label_path:
        data_to_load = data_path
        labels_to_load = label_path
        X = np.load(data_to_load)
        y = np.load(labels_to_load)
    else:
        data_to_load = f'./Datasets/kryptonite-{n}-X.npy'
        additional_data = f'./Datasets/additional-kryptonite-{n}-X.npy'

        labels_to_load = f'./Datasets/kryptonite-{n}-y.npy'
        additional_labels = f'./Datasets/additional-kryptonite-{n}-y.npy'
        if n >= 24:
            X = np.vstack((np.load(data_to_load), np.load(additional_data)))
            y = np.vstack((np.array(np.load(labels_to_load)).reshape(-1, 1),
                           np.array(np.load(additional_labels)).reshape(-1, 1)))
        else:
            X = np.load(data_to_load)
            y = np.load(labels_to_load)
    return X, y

In [38]:
def normalize_and_split_data(X, y):
    binary_X = (X >= 0.5).astype(int)
    X_train, X_test, y_train, y_test = train_test_split(binary_X, y, test_size=0.2, stratify=y)
    return X_train, X_test, y_train, y_test

In [44]:
def create_model(num_features):
    dropout = 0.05
    l2_reg = 5e-6
    model = tf.keras.Sequential([
        Input(shape=(num_features,)),
        Dense(128, activation='relu', kernel_regularizer=regularizers.L2(l2_reg), kernel_initializer=HeNormal()),
        BatchNormalization(),
        Dense(32, activation='relu', kernel_regularizer=regularizers.L2(l2_reg)),
        Dropout(dropout),
        Dense(16, activation='relu', kernel_regularizer=regularizers.L2(l2_reg)),
        Dropout(dropout),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [40]:
def do_cross_validation(X_train, y_train):
    fold_counter = 0
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=80)
    accuracies = []
    for train, val in kfold.split(X_train, y_train):
        model = create_model(X_train.shape[1])
        trained_model = model.fit(X_train[train], y_train[train], epochs=num_epochs, batch_size=batch_size, verbose=1, validation_data=(X_train[val], y_train[val]))
        scores = model.evaluate(X_train[val], y_train[val], verbose=0)
        accuracies.append(scores[1])
        print(f"Fold {fold_counter} Accuracy: {scores[1] * 100:.2f}%")
        fold_counter += 1

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(trained_model.history['accuracy'], label='Train Accuracy')
    plt.plot(trained_model.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(trained_model.history['loss'], label='Train Loss')
    plt.plot(trained_model.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.show()

    average_accuracy = np.mean(accuracies)
    print(f"Average Accuracy: {average_accuracy: .4f}")


In [56]:
X, y = load_data(18)
X_train, _, y_train, _ = normalize_and_split_data(X, y)
do_cross_validation(X_train, y_train)

Epoch 1/200
 88/360 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5066 - loss: 0.7679

2024-11-25 16:44:28.397553: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: condition [0], then [64,32], and else [] must be broadcastable
	 [[{{function_node __inference_one_step_on_data_20633717}}{{node sequential_102_1/dropout_204_1/stateless_dropout/SelectV2}}]]


InvalidArgumentError: Graph execution error:

Detected at node sequential_102_1/dropout_204_1/stateless_dropout/SelectV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/jackbrand/PycharmProjects/pythonProject/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/Users/jackbrand/PycharmProjects/pythonProject/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/Users/jackbrand/PycharmProjects/pythonProject/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/Users/jackbrand/PycharmProjects/pythonProject/.venv/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/Library/Frameworks/Python.framework/

In [41]:
def train_and_test_model(X_train, X_test, y_train, y_test, verbose=0):
    model = create_model(X_train.shape[1])
    model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, verbose=verbose)
    y_prediction_probabilities = model.predict(X_test).flatten()
    y_preds = (y_prediction_probabilities > 0.5).astype(int)

    accuracy = accuracy_score(y_test, y_preds)
    return accuracy

In [54]:
feature_dimensions = [9, 12, 15, 18, 24][::-1]
target_accuracies = [0.95, 0.925, 0.9, 0.875, 0.8][::-1]
# feature_dimensions = [9, 12, 15]
# target_accuracies = [0.95, 0.925, 0.9]
model_accuracies = []
for n in feature_dimensions:
    X, y = load_data(n)
    X_train, X_test, y_train, y_test = normalize_and_split_data(X, y)
    print(f"Beginning training n={n}")
    accuracy = train_and_test_model(X_train, X_test, y_train, y_test)
    model_accuracies.append(accuracy)
    print(f"Accuracy = {accuracy}")

# Assuming `feature_dimensions`, `model_accuracies`, and `target_accuracies` are predefined lists
data = pd.DataFrame({
    'Feature Dimensions': feature_dimensions * 2,
    'Accuracy': model_accuracies + target_accuracies,
    'Type': ['Model Accuracy'] * len(model_accuracies) + ['Target Accuracy'] * len(target_accuracies)
})

# Plot using Seaborn
sns.set(style='whitegrid')
plt.figure(figsize=(10, 6))
plot = sns.lineplot(
    data=data, x='Feature Dimensions', y='Accuracy', hue='Type',
    marker='o', style='Type', palette='pastel'
)

# Add data labels
for x, y, label in zip(data['Feature Dimensions'], data['Accuracy'], data['Accuracy']):
    plt.text(x, y, f'{label:.2f}', fontsize=10, ha='right', va='bottom')

# Customize the plot
plt.title('Model Accuracies vs Target Accuracies Across Feature Dimensions')
plt.xlabel('Feature Dimensions of Kryptonite-n (n)')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

Beginning training n=24


2024-11-25 16:16:55.098650: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: condition [0], then [64,32], and else [] must be broadcastable
	 [[{{function_node __inference_one_step_on_data_19077109}}{{node sequential_97_1/dropout_194_1/stateless_dropout/SelectV2}}]]


InvalidArgumentError: Graph execution error:

Detected at node sequential_97_1/dropout_194_1/stateless_dropout/SelectV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/jackbrand/PycharmProjects/pythonProject/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/Users/jackbrand/PycharmProjects/pythonProject/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/Users/jackbrand/PycharmProjects/pythonProject/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/Users/jackbrand/PycharmProjects/pythonProject/.venv/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/Library/Frameworks/Python.framework/V